# Chapter 2 - Model Selection and Training
Guilherme de Oliveira <br>
9/9/2016

## Introduction

In Chapter 2 we will work on the classification model of the US Census data that was analyzed in Chapter 1. My biggest interest in modelling will be dealing with the class imbalance of the target variable. In particular, I am interested in the following aspects:
<ul>
<li> How best to assess the accuracy of the classifier. It is unlikely that accuracy will suffice, because of the [accuracy paradox](https://en.wikipedia.org/wiki/Accuracy_paradox).
<li> What are some approaches that we can use to deal with the class imbalance? Examples include oversampling, undersampling, incorporating clustering algorithms, etc...
</ul>
<br>
<br>
<br>
# This is a work in progress. Stay tuned for more...
<br>
<br>
<br>


In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.grid_search import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

%matplotlib inline

## Preprocessing Data

In [ ]:
# preprocessing function - developped in Chapter 1
the_columns  = [('age', 'continuous'), 
                ('class_of_worker', 'nominal'), 
                ('detailed_industry_code', 'nominal'), 
                ('detailed_occupation_code', 'nominal'), 
                ('education', 'nominal'), 
                ('wage_per_hour', 'continuous'), 
                ('enrolled_in_edu_last_week', 'nominal'),
                ('marital_status', 'nominal'),
                ('major_industry_code', 'nominal'),
                ('major_occupation_code', 'nominal'),
                ('race', 'nominal'),
                ('hispanic_origin', 'nominal'),
                ('sex', 'binary'), # binary column with values Male/Female
                ('member_of_labor_union', 'nominal'), 
                ('reason_for_unemployment', 'nominal'),
                ('full_or_part_time_employment_stat', 'nominal'),
                ('capital_gains', 'continuous'),
                ('capital_losses', 'continuous'),
                ('dividends', 'continuous'),
                ('tax_filer', 'nominal'),
                ('region_of_previous_residence', 'nominal'),
                ('state_of_previous_residence', 'nominal'),
                ('detailed_household_family_stat', 'nominal'),
                ('detailed_household_summary', 'nominal'),
                ('instance_weight', 'IGNORE'), # as per instructions, to be dropped
                ('migration_code_change_in_msa', 'nominal'),
                ('migration_code_change_in_reg', 'nominal'),
                ('migration_code_move_within_reg', 'nominal'),
                ('live_in_this_house_1_yr_ago', 'nominal'),
                ('migration_prev_res_in_sunbelt', 'nominal'),
                ('num_persons_worked_for_employer', 'continuous'),
                ('family_members_under_18', 'nominal'),
                ('cob_father', 'nominal'),
                ('cob_mother', 'nominal'),
                ('cob_self', 'nominal'),
                ('citizenship', 'nominal'),
                ('own_business_or_self_employed', 'nominal'),
                ('fill_in_questionnaire_for_veterans_admin', 'nominal'),
                ('veterans_benefits', 'nominal'),
                ('weeks_worked_in_year', 'nominal'),
                ('year', 'nominal'), 
                ('savings','target')] # binary TARGET variable


In [ ]:
def preprocessData(file_name):
    # the_columns stores tuples of (column_name and tag for continuous/nominal/binary/target)
    
    raw_data = pd.read_csv(file_name, names=[c[0] for c in the_columns], index_col=False)
    original_shape = raw_data.shape
    
    raw_data.drop('instance_weight', axis=1, inplace=True)
    the_columns.remove(('instance_weight', 'IGNORE'))
    
    # find the duplicate rows, keep the first one
    duplicate_rows = raw_data.duplicated(keep='first')
    
    print 'number of duplicates = {:d}'.format(duplicate_rows.sum())
    raw_data = raw_data.drop_duplicates(keep='first')
    new_shape =  raw_data.shape
    print 'number of duplicates removed = {:d}'.format(original_shape[0] - new_shape[0])
    print 'original shape = {:d}, {:d}'.format(original_shape[0], original_shape[1])
    print 'new shape = {:d}, {:d}'.format(raw_data.shape[0], raw_data.shape[1])
    
    # convert nominal columns (object dtype) to integer type
    data = pd.DataFrame(raw_data.select_dtypes(include=['object']))
    object_columns = data.columns
    
    for column in object_columns:
        unique_values = data[column].unique()
        dictionary = {key:idx for idx,key in enumerate(unique_values)}
        data[column] = data[column].apply(lambda x : dictionary[x])
    
    # add nominal columns that were already in integer format 
    nominal_integer_columns = [c[0] for c in the_columns 
                               if c[1] == 'nominal' and c[0] not in data.columns]
    data[nominal_integer_columns] = raw_data[nominal_integer_columns]
    
    # convert 'sex', and 'savings' columns to binary; add year column
    data['savings'] = raw_data['savings'].map(lambda x: 
                                              1 if str(x).strip() == '50000+.' else 0)
    data['sex'] = raw_data['sex'].map(lambda x: 
                                      1 if str(x).strip() == 'Male' else 0)
    data['year'] = raw_data['year']
    
    # add continuous columns
    continuous_columns = [c[0] for c in the_columns if c[1] == 'continuous']
    data[continuous_columns] = raw_data[continuous_columns]
    
    # verify that we aren't missing any columns
    assert set(data.columns) == (set(raw_data.columns))
    
    text = 'The final processed data has {:,d} rows and {:d} columns.\n'
    print text.format(data.shape[0], data.shape[1])
    return data


In [ ]:
data = preprocessData('us_census_full/census_income_learn.csv')

In [ ]:
data.head(3)

## One-Hot Encoded Data
This will be required for logistic regression.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def one_hot_encode_nominal_columns():
    nominal_columns = [c[0] for c in the_columns if c[1] == 'nominal']
    
    dummy_columns = [pd.get_dummies(data[col], prefix=col, prefix_sep='.') 
                     for col in nominal_columns]
    
    one_hot_encoded_data = pd.concat(dummy_columns, axis=1)
    print '\nThere were {:d} nominal columns to be converted.'.format(len(nominal_columns))
    print 'The number of one-hot-encoded columns is {:d}.\n'.format(data.shape[1])
    
    # check size
    count_distinct_values = 0
    for column in nominal_columns:
        count_distinct_values += len(data[column].unique())
    
    assert count_distinct_values == one_hot_encoded_data.shape[1], \
        "mismatch between number of dummy columns and unique values"
    
    return one_hot_encoded_data

In [ ]:
ohe_data = one_hot_encode_nominal_columns()

# add target (savings)
ohe_data['savings'] = data['savings']

# scale and add continuous columns
min_max_scaler = MinMaxScaler()
continuous_cols = [c[0] for c in the_columns if c[1] == 'continuous']
ohe_data[continuous_cols] = pd.DataFrame(min_max_scaler.fit_transform(
        data[continuous_cols]), columns=continuous_cols, index = data.index)

print 'The final shape is: {:,d} x {:d}.'.format(ohe_data.shape[0], ohe_data.shape[1])
mx = ohe_data.max().max()
mn = ohe_data.min().min()
print 'To verify scaling: max = {:.2f}, min={:.2f}\n'.format(mx, mn)

## Functions for Modelling

In [ ]:
def get_train_test_data_sets(X, y):
    # obtain training and test set for cross-validation

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    print 'size of training data: {:7d}, {:3d}'.format(X_train.shape[0], X_train.shape[1])
    print 'size of test data:     {:7d}, {:3d}'.format(X_test.shape[0], X_test.shape[1])
    print
    ytr0, ytr1 = (y_train == 0).sum(), (y_train == 1).sum()
    yte0, yte1 = (y_test == 0).sum(), (y_test == 1).sum()
    print 'y_train==0: {:6d},  y_train==1: {:4d},  balance: {:.4f}'.format(
        ytr0, ytr1, float(ytr0)/(ytr0+ytr1))
    print 'y_test==0:  {:6d},  y_test==1:  {:4d},  balance: {:.4f}'.format(
        yte0, yte1, float(yte0)/(yte0+yte1))
    print '\nThe training and test set appear to have the same degree of class imbalance.\n'
    
    return X_train, X_test, y_train, y_test

In [ ]:
def print_confusion_matrix(y_true, y_pred):
    header = '\t          prediction 0    prediction 1'
    row0 =   '\tclass 0 {:11,d} {:14,d}'
    row1 =   '\tclass 1 {:11,d} {:14,d}'
    cm = confusion_matrix(y_true, y_pred)
    print header
    print row0.format(cm[0,0], cm[0,1])
    print row1.format(cm[1,0], cm[1,1])
    tp, fn = float(cm[0,0]), float(cm[0,1])
    fp, tn = float(cm[1,0]), float(cm[1,1])
#     print 'precision = {:.4f},  {:.4f}'.format(tp/(tp+fp), tn/(tn+fn))
#     print 'recall =    {:.4f},  {:.4f}'.format(tp/(tp+fn), tn/(tn+fp))


In [ ]:
def run_grid_search(classifier, parameters, score=None, print_grid_scores=False, verbose=0):
    
    rf_clf = GridSearchCV(classifier, 
                          parameters, 
                          scoring=score,
                          verbose=verbose)
    
    rf_clf.fit(X_train, y_train)
    y_pred = rf_clf.predict(X_test)
    
    print 'Best parameters on training set:'
    print rf_clf.best_params_
    print '\nBest score = {:.4f}'.format(rf_clf.best_score_)
    if print_grid_scores:
        print '\nGrid scores on training set:\n'
        for params, mean_score, scores in rf_clf.grid_scores_:
            print("%0.4f (+/-%0.04f) for %r"
                  % (mean_score, scores.std() * 2, params))
    print
    print 'confusion matrix'
    print_confusion_matrix(y_test, y_pred)
    print '\nDetailed classification report:'
    print classification_report(y_test, y_pred, digits=5)
    return rf_clf

In [ ]:
def plot_roc_curves(models, scores):
    plt.figure(figsize=(8,8))
    ax = plt.gca()
    fs=14
    
    for rf, label in zip(models,scores):
        probs = rf.predict_proba(X_test)
        fpr, tpr, thresholds = roc_curve(y_test, probs[:, 1])
        roc_auc = auc(fpr, tpr)
        ax.plot([0, 1], [0, 1], 'k--')
        text = ' (area = {:.3f})'.format(roc_auc)
        ax.plot(fpr, tpr, label=label + text)
    
    ax.set_xlabel('False positive rate', fontsize=fs)
    ax.set_ylabel('True positive rate', fontsize=fs)
    ax.set_title('ROC curve', fontsize=fs)
    ax.set_xticks(np.arange(0.0, 1.01, 0.2))
    ax.set_xticklabels(np.arange(0.0,1.01,0.2), fontsize=fs)
    ax.set_yticks(np.arange(0.0, 1.01, 0.2))
    ax.set_yticklabels(np.arange(0.0,1.01,0.2), fontsize=fs)
    ax.set_ylim((0,1.01))
    ax.set_xlim((-0.01, 1.0))
    plt.legend(fontsize=fs, bbox_to_anchor=(1.05,0.5), loc='center left')
    plt.show()

## Random Forest Model
Use scikit-learn modules to run a grid search for a random forest model to find the best parameters. Three cases are run, optimized on different scores: accuracy, precision and recall. Confusion matrices are displayed as well as a classification report.

In [ ]:
X = data.drop('savings', axis=1)
y = data.loc[:,'savings']

X_train, X_test, y_train, y_test = get_train_test_data_sets(X, y)


In [ ]:
parameters = {'n_estimators': [20, 40], 
              'max_depth': [6, None],
              'max_features' : ['sqrt', 40],
              'min_samples_split' : [1, 2]}

parameters = {'n_estimators': [10], 'max_depth': [6], 'max_features' : ['sqrt'], 'min_samples_split' : [1]}


In [ ]:
rf_a = run_grid_search(RandomForestClassifier(), parameters,
                       score='accuracy', verbose=1)

In [ ]:
rf_p = run_grid_search(RandomForestClassifier(), parameters,
                       score='precision', verbose=1)

In [ ]:
rf_r = run_grid_search(RandomForestClassifier(), parameters,
                       score='recall', verbose=1)

In [ ]:
plot_roc_curves([rf_a, rf_p, rf_r],['accuracy', 'precision', 'recall'])

## Logistic Regression
Use one-hot encoded data.

In [ ]:
X = ohe_data.drop('savings', axis=1)
y = ohe_data.loc[:,'savings']

X_train, X_test, y_train, y_test = get_train_test_data_sets(X, y)


In [ ]:
parameters = {'penalty': ['l1', 'l2'], 'C': [1.0, 0.5, 0.1, 0.05, 0.01]}

parameters = {'penalty': ['l1', 'l2'], 'C': [0.1]}

In [ ]:
lr_a = run_grid_search(LogisticRegression(), score='accuracy', parameters,
                       verbose=1, print_grid_scores=True)

In [ ]:
lr_p = run_grid_search(LogisticRegression(), score='precision', parameters,
                       verbose=1, print_grid_scores=True)

In [ ]:
lr_r = run_grid_search(LogisticRegression(), score='recall', parameters,
                       verbose=1, print_grid_scores=True)

In [ ]:
plot_roc_curves([lr_a, lr_p, lr_r],['accuracy', 'precision', 'recall'])

## Incorporate Some Feature Engineering
Start with the column "detailed_household_family_stat" and convert the classes that have no savings greater than 50K into one class.

In [ ]:
def update_column(column):
    dhfs = data[column][data['savings'] == 1].unique()
    dhfs.sort()
    print 'unique values for svngs = 1', dhfs
    dhfs_all = data[column].unique()
    dhfs_all.sort()
    print 'unique values for all vals ', dhfs_all
    
    diff = set(dhfs_all).difference(set(dhfs))
    print ' the differences are........', diff
    if diff is None:
        print '\n diff is empty'
        return data[column]
    
    print ' len(diff)', len(diff)
    
    val = max(diff) + 1
    print ' mapping values to:', val
    return data[column].map(lambda x : val if x in diff else x)


In [ ]:
print data['detailed_household_family_stat'][data['savings']==0].value_counts().shape
data['detailed_household_family_stat'] = update_column('detailed_household_family_stat')
print data['detailed_household_family_stat'][data['savings']==0].value_counts().shape


In [ ]:
print data['family_members_under_18'][data['savings']==0].value_counts().shape
data['family_members_under_18'] = update_column('family_members_under_18')
print data['family_members_under_18'][data['savings']==0].value_counts().shape


In [ ]:
rf_clf = RandomForestClassifier(n_estimators = 40, max_depth=None, max_features=40, min_samples_split=1)
rf_clf.fit(X_train, y_train)

y_pred = rf_clf.predict(X_test)

print 'confusion matrix:'
print_confusion_matrix(y_test, y_pred)
print '\nDetailed classification report:'
print classification_report(y_test, y_pred, digits=5)

In [ ]:
# Fit and transform x to visualise inside a 2D feature space
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
data_vis = pca.fit_transform(X)
print 'shape(data_vis):', data_vis.shape
print data_vis[:4,:]
print data_vis[-4:,:]
print 'pca.components_.shape:', pca.components_.shape
print 'pca.explained_variance_ratio_:', pca.explained_variance_ratio_

In [ ]:
print 'y==0 : ', (y==0).sum()
print 'y==1 : ', (y==1).sum()
print 'y==0 + y==1:', (y==0).sum() + (y==1).sum()

In [ ]:
# Plot the original data
# Plot the two classes

def scatter_plot(X, y):
    data_vis = pca.fit_transform(X)
    yeq0 = data_vis[ (y==0) ]
    yeq1 = data_vis[ (y==1) ]
    palette = sns.color_palette()
    almost_black = '#262626'
    fig=plt.figure(figsize=(9,9));
    ax = fig.gca();
    ax.scatter(yeq0[:, 0], yeq0[:, 1], label="Savings < 50K", alpha=0.3, facecolor=palette[0], 
               linewidth=0.15, edgecolor=almost_black);
    ax.scatter(yeq1[:, 0], yeq1[:, 1], label="Savings > 50K", alpha=0.3, facecolor=palette[2], 
               linewidth=0.15, edgecolor=almost_black);
    ax.legend(fontsize=16, loc='lower left', bbox_to_anchor=(1,0.8));
    return ax

In [ ]:
ax = scatter_plot(X, y);

## Class Imblance: Under and Over Sampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
# Generate the new dataset using under-sampling method
verbose = False

# 'Random under-sampling'
US = RandomUnderSampler()
usx, usy = US.fit_sample(X, y)
ax = scatter_plot(usx, usy);
